# 손글씨 숫자 인식

In [5]:
# Library Import
import numpy as np

## MNIST 손글씨 Dataset
- 기계학습 분야에서 사용되는 유명한 데이터셋
- 0~9 숫자 이미지로 구성
- 훈련 이미지 : 60,000장 , 테스트 이미지 : 10,000장

In [2]:
import sys,os
sys.path.append(os.pardir)
from dataset.mnist import load_mnist

(x_train,y_train) , (x_test,y_test) = load_mnist(normalize=True,flatten=True,one_hot_label=True)

print("x_train :",x_train.shape) # 학습 이미지 
print("x_test :",x_test.shape) # 테스트 이미지
print("y_train :",y_train.shape) # 학습 이미지 abel
print("y_test :",y_test.shape) # 테스트 이미지 label

x_train : (60000, 784)
x_test : (10000, 784)
y_train : (60000, 10)
y_test : (10000, 10)


## 신경망의 추론 처리
> **본 Part에서는 학습 구현은 이루어지지 않고 추론을 수행하는 구현만 이루어집니다.**
- `get_data()` : MNIST 데이터셋을 반환합니다.
  - 본 Part는 추론 과정만 구현함으로 학습데이터는 반환하지 않고 테스트데이터만 반환합니다.
- `init_network()` : 신경망의 Weight와 bias 값을 정의합니다.
  - 본 Part는 미리 구현된 pickle 파일을 불러옵니다.
- `predict()` : 신경망의 추론과정 즉 순전파(forward propagation)를 수행합니다.

- 신경망의 구성
  - 입력층 노드 : 784
  - 첫번째 은닉층 노드 : 50
  - 두번째 은닉층 노드 : 100
  - 출력층 노드 : 10 (0~9까지 10개의 클래스)

In [15]:

def get_data():
    (x_train,y_train),(x_test,y_test) = load_mnist(normalize=True,flatten=True,one_hot_label=False)
    return x_test,y_test # 테스트데이터만 return 

import pickle
def init_network():
    with open("sample_weight.pkl",'rb') as f:
        network = pickle.load(f)
    return network


def predict(network,x):
    W1,W2,W3 = network['W1'] , network['W2'] , network['W3']
    b1,b2,b3 = network['b1'] , network['b2'] , network['b3']

    a1 = np.dot(x,W1) + b1
    z1 = sigmoid(a1)
    a2 = np.dot(z1,W2) + b2
    z2 = sigmoid(a2)
    a3 = np.dot(z2,W3) + b3
    y = softmax(a3)

    return y 
    

`sample_weight.pkl` 파일에서 불러온 weight(편의상 bias와 합쳐서 부르겠습니다.)가 어떤 shape을 가지고 있고 우리가 구현하려는  
신경망이 맞는지 확인해보도록 하겠습니다.  
우리가 구현하려는 신경망은 첫번째 은닉층 계층의 노드가 50 두번째 은닉층 계층의 노드가 100 으로 구성되어 있고 최종 출력층 노드가 10개 입니다.

In [19]:
with open('sample_weight.pkl','rb') as f:
    network = pickle.load(f)

print('첫번째 은닉층 계층 weight :',network['W1'].shape)
print('첫번째 은닉층 계층 bias :',network['b1'].shape)
print('두번째 은닉층 계층 weight :',network['W2'].shape)
print('두번째 은닉층 계층 bias :',network['b2'].shape)
print('출력층 weight :',network['W3'].shape)
print('출력층 bias :',network['b3'].shape)

첫번째 은닉층 계층 weight : (784, 50)
첫번째 은닉층 계층 bias : (50,)
두번째 은닉층 계층 weight : (50, 100)
두번째 은닉층 계층 bias : (100,)
출력층 weight : (100, 10)
출력층 bias : (10,)


구현을 하다보니 활성화 함수와 출력층의 활성화 함수를 구현하지 않았습니다. 에러가 발생할 수 있으니 구현하도록 하겠습니다.

In [16]:
def sigmoid(x): # 은닉층 활성화 함수 Sigmoid
    return 1 / (1+np.exp(-x))

def softmax(x): # 출력층 활성화 함수 Softmax
    return np.exp(x) / np.sum(np.exp(x))


활성화 함수까지 모두 구현해 주었으니 본격적으로 신경망을 구현하기 전 준비는 끝났습니다. 그렇다면 지금부터 신경망을 구현하여 추론과정을 진행하고.  
추론과정에서 나온 결과를 토대로 실제 label 값과 비교하여 Accuracy까지 구해보도록 하겠습니다.

In [17]:
x,y = get_data() # 추론과정만 구현했으므로 테스트데이터만 받아옵니다
network = init_network() # 미리 구한 weight와 bias 피클 파일을 불러옵니다.

correct_cnt = 0 # 모델이 추론한 값과 실제 값이 동일한 것에 개수를 저장하는 변수

for i in range(len(x)):
    y_hat = predict(network,x[i])
    if np.argmax(y_hat) == y[i]:
        correct_cnt +=1
    
print('Accuracy :',float(correct_cnt / len(x)))


Accuracy : 0.9352


Accuracy 가 약 93% 정도가 나오는걸 확인할 수 있습니다. 이렇게 추론과정까지 구현해보았습니다. 그렇다면 이젠 배치학습을 
구현해보도록 하겠습니다. 배치란 데이터의 묶음 이라고 생각하면 됩니다. 배치 크기를 100으로 해보겠습니다.(1 배치당 100개 데이터)  
우리의 데이터는 numpy 배열로 구현되어있기 때문에 행렬곱을 통한 연산이 가능합니다!

In [21]:
x,y = get_data()
network = init_network()
batch_size = 100

current_cnt = 0

for i in range(0,len(x),batch_size):
    x_batch = x[i:i+batch_size]
    y_batch = predict(network,x[i:i+batch_size])
    p = np.argmax(y_batch,axis=1)
    current_cnt += np.sum(p == y[i:i+batch_size])

print('Accuracy :',float(current_cnt / len(x)))

Accuracy : 0.9352
